# * Import : "Metric Master List"

## Parameter

In [9]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd

# Input parameter
src_dir = 'data'
src_file = 'Metric Master List'
src_sheet = 'DIM_METRIC_DATA'
curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Rawdata

In [27]:
data_src = f'{src_dir}/{src_file}.xlsx'
src_df = pd.read_excel(data_src, sheet_name=src_sheet, index_col=None) 
# , skiprows=2, usecols='N:AD'
src_df.head(3)

,METRIC_CD,METRIC_NAME,METRIC_ID,COMP_CD,CENTER,AGG_TYPE,RR_IND,CONS_IND,METRIC_GRP,METRIC_SUB_GRP,...,Network,Sales,Experience,Mobile,Prepaid,Postpaid,TOL,TVS,TDG,B2B
0,B0R000100,Total Revenue,B0R000100,ALL,Revenue,S,1.0,N,Revenue,Revenue,...,No,Yes,No,No,No,No,No,No,No,No
1,B0R00010001GEOS,Total Inflow M1 (Geo),B0R00010001,ALL,Sales,S,1.0,N,Sales,Sales,...,No,Yes,No,No,No,No,No,No,No,No
2,B0R00010001,Total Inflow M1,B0R00010001,ALL,Revenue,S,1.0,S,Sales,Sales,...,No,Yes,No,No,No,No,No,No,No,No


In [35]:
cols = ['METRIC_CD', 'METRIC_NAME', 'METRIC_ID', 'COMP_CD', 'CENTER', 'AGG_TYPE', 'RR_IND', 'CONS_IND', 'METRIC_GRP', 'METRIC_SUB_GRP', 'PRODUCT_GRP', 'GRY_IND', 'UOM', 'DEF', 'CAL', 'CORPORATE', 'MICRO_ORG', 'METRIC_TYPE', 'RADAR', 'KPI_TYPE', 'LIST', 'REMARK', 'Data Source']
tmp_df = src_df[cols]

# Filter
tmp_df = tmp_df.loc[tmp_df['CENTER'].isin(['Revenue', 'Sales', 'Profitability'])]
tmp_df = tmp_df.loc[~tmp_df['METRIC_CD'].str.contains('[0-9]C$|[0-9]H$|[0-9]MCOM$|[0-9]CUS$')]
tmp_df = tmp_df.loc[~tmp_df['METRIC_NAME'].str.contains('Geo')]
tmp_df = tmp_df.loc[tmp_df['PRODUCT_GRP']=='TVS']
# src_df = src_df.replace(np.nan, None)
tmp_df = tmp_df.reset_index(drop=True)

tmp_df.tail()

,METRIC_CD,METRIC_NAME,METRIC_ID,COMP_CD,CENTER,AGG_TYPE,RR_IND,CONS_IND,METRIC_GRP,METRIC_SUB_GRP,...,DEF,CAL,CORPORATE,MICRO_ORG,METRIC_TYPE,RADAR,KPI_TYPE,LIST,REMARK,Data Source
107,TSER14102,TVS Churn Subs Voluntary,SER14102,True,Revenue,S,1.0,N,Retention & Churn,Churn,...,Actual voluntary churn subs per day,(Number of churn subs),Yes,Yes,NaN,NaN,NaN,NaN,NaN,TVS
108,TSER14103,TVS Churn Subs Involuntary,SER14103,True,Revenue,S,1.0,N,Retention & Churn,Churn,...,Actual involuntary churn subs per day,(Number of churn subs),Yes,Yes,NaN,NaN,NaN,NaN,NaN,TVS
109,TSER14201,TVS Churn Rate Voluntary,SER14201,True,Revenue,N,0.0,N,Retention & Churn,Churn,...,Last 30 days Voluntary Churn rate per month of...,(Number of churn subs during the past 30 days)...,Yes,Yes,NaN,NaN,NaN,NaN,NaN,CEx
110,TSER14202,TVS Churn Rate Involuntary,SER14202,True,Revenue,N,0.0,N,Retention & Churn,Churn,...,Last 30 days Involuntary Churn rate per month ...,(Number of churn subs during the past 30 days)...,Yes,Yes,NaN,NaN,NaN,NaN,NaN,CEx
111,TSER41201,TVS Save Rate,SER41201,True,Revenue,N,0.0,N,Retention & Churn,Reactive Retention,...,Retention save rate from customer churn reques...,(number of saved subs) / (number of requested ...,Yes,Yes,NaN,NaN,NaN,NaN,NaN,CEx


## Generate Output file

In [36]:
# to Excel file

op_dir = 'temp'
op_file = 'TMP_Metric Master List.xlsx'

tmp_df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
print(f'\n  -> Generate "{op_file}" successfully')


  -> Generate "TMP_Metric Master List.xlsx" successfully
